In [1339]:
import pandas as pd
import numpy as np
import re
import requests 
import bs4 

from bs4 import BeautifulSoup
from user_agent import generate_user_agent

headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac'))} 

In [1064]:
movie_years = [str(year) for year in range(1997, 2018)]
movie_links = []

for year in movie_years:
    r = requests.get('https://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr=%s&p=.htm' %year, headers=headers)
    r_text = BeautifulSoup(r.text, 'html.parser')
    a_tags = r_text.find_all('a')
    all_links_year = [link.get('href') for link in a_tags if link.get('href') is not None] # Getting hyperlinks
    movie_links_year = [("https://www.boxofficemojo.com" + movie) for movie in all_links_year if '/movies/?' in movie] # Filtering hyperlinks for ones which link to movies
    movie_links_year = movie_links_year[1:] #First entry is usually incorrect
    for link in movie_links_year:
        movie_links.append(link)
movie_links = [movie.replace("elizabeth\xa0.htm", "elizabeth%A0.htm") if "elizabeth\xa0.htm" in movie else movie for movie in movie_links] #Quick solution to deal with a single annoying anomaly

In [1229]:
# Function to scrape box office data from Boxoffice Mojo 
def scrape_boxoffice(movie_list):
    movie_data = [] #Creating a list that will store more lists, one per movie
    for link in movie_list:
        indiv_movie = []
        r_link = requests.get(link)
        r_link_text = BeautifulSoup(r_link.text, 'html.parser')
        table = r_link_text.find_all('table') #Information is contained in tables
        try: 
            table_td = table[2].find_all('td')
            for ii in table_td:
                if ii.find('td') != None:
                    indiv_movie.append(ii.find('td').contents[0])
                if ii.find('b')!= None:
                    indiv_movie.append(ii.find('b').contents[0])
                if ii.find('font') != None:
                    indiv_movie.append(ii.find('font').contents[0])
                if ii.find('th') != None:
                    indiv_movie.append(ii.find('tr').contents[0])
            clean_title = indiv_movie[2].text.replace(':', " ") # Removing :, (, and ) from the movie titles. Will use grep in the future
            clean_title = clean_title.replace("(", "")
            clean_title = clean_title.replace(")", "")
            information_list = [clean_title]
            try:
                for ii in [9, 13, 16]: # 9, 13 and 16 are indexes for Domestic Gross, Genre and Production Budget
                    information_list.append(str(indiv_movie[ii])) #Convert NavigableStrings to string
                information_list.append(indiv_movie[12].text) # Release date
            except IndexError: 
                pass
            movie_data.append(information_list)
        except IndexError:
            pass

    db = pd.DataFrame(movie_data, columns = ['Title', 'Domestic Gross', 'Genre',  'Budget', 'Release Date']) #Dataframe from list of lists
    return(db)
    
movie_data_df = scrape_boxoffice(movie_links)

In [1374]:
titles = [] 
for ii in movie_data_df['Title']:
    titles.append(ii.replace(" ", "+"))#Removing spaces in order to generate valid urls for OMDB database
titles[2430] = 'awesome%3B+i+fuckin+shot+that' #Fixing a tricky formatting issue directly

'awesome%3B+i+fuckin+shot+that'

In [1436]:
# Requesting OMDB data via API; broken into parts
omdb_json_list=[]
for title in titles: 
    r_omdb = requests.get("http://www.omdbapi.com/?apikey=d295246e&t=%s" %title) 
    if len(r_omdb.json()) > 3: # To ignore missing movies
        omdb_json_list.append(r_omdb.json())  #Each JSON contains information; append them 
    else: 
        pass

In [1442]:
# Extracting data from OMDB movies
omdb_movie_list = [] 
for movie in omdb_json_list: 
    try:
        dataframe_variables = [movie['Title'], movie['Year'], movie['Rated'], movie['Runtime'], movie['Metascore'], movie['imdbRating'], movie['Ratings'][1]['Value']] 
        omdb_movie_list.append(dataframe_variables) 
    except: pass #Some movies not found

In [1506]:
# Joining Data  
omdb_movie_data = pd.DataFrame(omdb_movie_list, columns = ['Title', 'Year', 'Rating', 'Runtime', 'Metascore', 'IMDB score', 'RT score'])
omdb_movie_data['Title'] = omdb_movie_data['Title'].str.replace(":","") # Cleaning title column to allow for merge
merged = movie_data_df.merge(omdb_movie_data, left_on="Title", right_on="Title", how="outer") # Outer join to ensure that all data is present

In [1536]:
# Cleaning
merged = merged.replace('N/A', np.nan) # Convert N/A to NaN
merged = merged.replace([None], np.nan, regex=True) # Replace None values with Nan to allow operations or iteration

In [1549]:
raw_boxoffice_df = merged.copy()
raw_boxoffice_df['Domestic Gross'] = raw_boxoffice_df['Domestic Gross'].str.extract('(\d+)').astype(float) #Str to Float
raw_boxoffice_df['Budget'] = raw_boxoffice_df['Budget'].str.extract('(\d+)').astype(float) # Keep digits from str, save as float
raw_boxoffice_df['Release Date'] = raw_boxoffice_df['Release Date'].astype(str) # Convert to str
raw_boxoffice_df['Runtime'] = raw_boxoffice_df['Runtime'].str.replace(" min", "").astype(float) # Convert minutes to float
raw_boxoffice_df['Metascore'] = raw_boxoffice_df['Metascore'].astype(float) # Convert to type float
raw_boxoffice_df['RT score'] = raw_boxoffice_df['RT score'].str.replace("%", "").str.replace("/100", "").astype(float)
raw_boxoffice_df.columns = ['Title', 'Domestic Gross (millions)', 'Genre', 'Budget (millions)', 'Release Date', 'Year', 'Rating', 'Runtime', 'Metascore', 'IMDB score', 'RT score']

# New Columns
raw_boxoffice_df['Release Month'] = raw_boxoffice_df['Release Date'].str.split().map(lambda x: x[0]) #Establishing month of release
raw_boxoffice_df['Domestic ROI Multiplier'] = raw_boxoffice_df['Domestic Gross (millions)'].fillna(0)/raw_boxoffice_df['Budget (millions)'].fillna(0) #Diving the two columns to get multiplier; fillna to prevent NaN errors
raw_boxoffice_df['Domestic ROI Multiplier'] = raw_boxoffice_df['Domestic ROI Multiplier'].replace([np.inf], np.nan) #repliace Inf values with Nan 

In [1554]:
raw_boxoffice_df.columns

Index(['Title', 'Domestic Gross (millions)', 'Genre', 'Budget (millions)',
       'Release Date', 'Year', 'Rating', 'Runtime', 'Metascore', 'IMDB score',
       'RT score', 'Release Month', 'Domestic ROI Multiplier'],
      dtype='object')

In [1551]:
# Write data to local csv
raw_boxoffice_df.to_csv('/Users/benisonp/Documents/Data Science/raw_boxoffice_df.csv')